# Bank Loan Prediction Project

## Project Description

This project aims to predict whether a customer will take a personal loan based on their demographic and financial data using machine learning models.

### a. General Information on Dataset
- **Dataset Name**: Bank Loan Dataset (bankloan.csv)


Target Variable:
Personal Loan (binary numerical variable ∈ {0,1})

Problem Type:
Supervised regression task where the objective is to estimate a continuous
loan acceptance score.

Number of Classes:
Not applicable (regression problem)

Total Number of Samples:
5000 records 

Train / Validation / Test Split:
- Training: 4000 samples (80%)
- Validation: Performed via 5-Fold Cross-Validation on training data
- Testing: 1000 samples (20%)


### b. Implementation Details
- **Feature Extraction**: 13 features extracted from the dataset after dropping the ID column
  - Features: Age, Experience, Income, ZIP Code, Family, CCAvg, Education, Mortgage, Securities Account, CD Account, Online, CreditCard
  - Dimension: (5000, 13)

- **Preprocessing**: StandardScaler applied to features for both models



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import resample

In [ ]:
df = pd.read_csv(r"E:\1st Semster\ML\Bank loan\dataset\bankloan.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
print("\nMissing Values:")
print(df.isna().sum())

In [ ]:
df.rename(columns={"Family": "family_member"}, inplace=True)

# Replace negative values in 'Experience' with 0
df["Experience"] = df["Experience"].apply(lambda x: 0 if x < 0 else x)

# Preview the result
df.head()

In [ ]:
df = df.drop(columns=["ID"])

In [ ]:
df_no_loan = df[df["Personal.Loan"] == 0]
df_loan = df[df["Personal.Loan"] == 1]


df_no_loan_downsampled = resample(
    df_no_loan,
    replace=False,
    n_samples=len(df_loan),
    random_state=42
)

# Combine to form balanced dataset
df_balanced = pd.concat([df_no_loan_downsampled, df_loan])

# Shuffle dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify balance
df_balanced["Personal.Loan"].value_counts()


In [ ]:
X = df.drop("Personal.Loan", axis=1)
y = df["Personal.Loan"]

X = df_balanced.drop("Personal.Loan", axis=1)  # ← USES df_balanced now!
y = df_balanced["Personal.Loan"]
print("Data Preprocessing Complete")



In [ ]:
# Cell 5: Dataset Information (Requirements Section A)
print("\n" + "="*70)
print("A. GENERAL INFORMATION ON DATASET")
print("="*70)
print("\nProject: Bank Loan Approval Prediction")
print("Dataset: Bank Loan Dataset (bankloan.csv)")
print("Task Type: Regression (predicting loan approval probability)")

# Show BOTH original and balanced
print(f"\nOriginal dataset: {len(df)} samples")
print(f"Balanced dataset: {len(df_balanced)} samples")
print(f"Target variable: Personal.Loan (0 = No loan, 1 = Loan approved)")




print("\n" + "="*70)
print("A. GENERAL INFORMATION ON DATASET")
print("="*70)
print("\nProject: Bank Loan Approval Prediction")
print("Dataset: Bank Loan Dataset (bankloan.csv)")
print("Task Type: Regression (predicting loan approval probability)")
print(f"\nTotal number of samples: {len(df_balanced)}")
print(f"Target variable: Personal.Loan (0 = No loan, 1 = Loan approved)")
print(f"Target distribution:\n{y.value_counts()}")
print("\nNote: This is a numerical dataset (tabular data)")


In [ ]:
# Cell 6: Feature Information
print("\n" + "="*70)
print("B. IMPLEMENTATION DETAILS - FEATURE EXTRACTION")
print("="*70)
print(f"\nTotal columns in dataset: {df_balanced.shape[1]} (includes target)")
print(f"Number of features: {X.shape[1]} (excluding target variable)")
print(f"\nFeature names: {list(X.columns)}")
print(f"Feature matrix shape: {X.shape}")
print(f"Feature matrix dimensions: {X.shape[0]} samples × {X.shape[1]} features")
print(f"Target variable shape: {y.shape}")

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Second split: 75% train, 25% validation (of the 80%)
# This gives us 60% train, 20% validation, 20% test overall
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

print("\n" + "="*70)
print("DATA SPLIT")
print("="*70)
print(f"Training samples: {len(X_train)} ({len(X_train)/len(df_balanced)*100:.1f}%)")
print(f"Validation samples: {len(X_val)} ({len(X_val)/len(df_balanced)*100:.1f}%)")
print(f"Testing samples: {len(X_test)} ({len(X_test)/len(df_balanced)*100:.1f}%)")
print(f"Total: {len(X_train) + len(X_val) + len(X_test)} samples")



In [ ]:
# Cell 8: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n" + "="*70)
print("FEATURE SCALING")
print("="*70)
print("Method: StandardScaler")
print("  - Standardizes features by removing mean and scaling to unit variance")
print("  - Formula: z = (x - mean) / std")
print("Scaling completed on train, validation, and test sets")



In [ ]:

# Cell 9: Cross-Validation Setup
print("\n" + "="*70)
print("CROSS-VALIDATION SETUP")
print("="*70)
print("Method: K-Fold Cross-Validation")
print("Number of folds: 5")
print("Training/Validation ratio per fold: 80/20 (4 folds train : 1 fold validation)")
print("Shuffle: True")
print("Random state: 42")
print("\nNote: Cross-validation is performed on the training set only")

kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
# Cell 10: Hyperparameters (Requirements Section B)
print("\n" + "="*70)
print("B. HYPERPARAMETERS")
print("="*70)

print("\n1. Linear Regression Hyperparameters:")
print("   - fit_intercept: True (model includes bias term)")
print("   - copy_X: True (default)")
print("   - n_jobs: None (single core)")
print("   - positive: False (coefficients can be negative)")
print("   - Note: No regularization applied")

print("\n2. KNN Regressor Hyperparameters:")
print("   - n_neighbors: 5 (number of nearest neighbors)")
print("   - weights: 'uniform' (all neighbors weighted equally)")
print("   - algorithm: 'auto' (automatically choose best algorithm)")
print("   - metric: 'minkowski' with p=2 (equivalent to Euclidean distance)")
print("   - leaf_size: 30 (default)")

print("\n3. Data Preprocessing Hyperparameters:")
print("   - Feature scaling: StandardScaler (mean=0, std=1)")
print("   - Train/Validation/Test split: 60/20/20")
print("   - Random state: 42 (for reproducibility)")

print("\n4. Not Applicable for These Models:")
print("   - Learning rate: N/A (not iterative optimization)")
print("   - Optimizer: N/A (closed-form solution for LR, instance-based for KNN)")
print("   - Batch size: N/A (not mini-batch training)")
print("   - Number of epochs: N/A (not iterative training)")
print("   - Regularization: N/A (basic models without regularization)")



In [ ]:
# Cell 11: Model Training
print("\n" + "="*70)
print("MODEL TRAINING")
print("="*70)

# Initialize models
lr = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=5)

# Train models
lr.fit(X_train_scaled, y_train)
knn.fit(X_train_scaled, y_train)

print("✓ Linear Regression trained successfully")
print("✓ KNN Regressor trained successfully")


In [ ]:
# Cell 12: Validation Set Performance
print("\n" + "="*70)
print("VALIDATION SET PERFORMANCE")
print("="*70)

# Predictions on validation set
y_val_pred_lr = lr.predict(X_val_scaled)
y_val_pred_knn = knn.predict(X_val_scaled)

print("\nLinear Regression - Validation Results:")
print(f"  MAE:  {mean_absolute_error(y_val, y_val_pred_lr):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_val, y_val_pred_lr)):.4f}")
print(f"  R²:   {r2_score(y_val, y_val_pred_lr):.4f}")

print("\nKNN Regressor - Validation Results:")
print(f"  MAE:  {mean_absolute_error(y_val, y_val_pred_knn):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_val, y_val_pred_knn)):.4f}")
print(f"  R²:   {r2_score(y_val, y_val_pred_knn):.4f}")


In [ ]:
# Cell 13: Test Set Predictions
# Make predictions on test set
y_pred_lr = lr.predict(X_test_scaled)
y_pred_knn = knn.predict(X_test_scaled)

print("\n" + "="*70)
print("C. RESULTS DETAILS - TEST SET PERFORMANCE")
print("="*70)

print("\nLinear Regression - Test Results:")
print(f"  MAE:  {mean_absolute_error(y_test, y_pred_lr):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.4f}")
print(f"  R²:   {r2_score(y_test, y_pred_lr):.4f}")

print("\nKNN Regressor - Test Results:")
print(f"  MAE:  {mean_absolute_error(y_test, y_pred_knn):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_knn)):.4f}")
print(f"  R²:   {r2_score(y_test, y_pred_knn):.4f}")


In [ ]:
# Cell 14: Cross-Validation Scores
print("\n" + "="*70)
print("CROSS-VALIDATION SCORES (5-Fold)")
print("="*70)

# Combine train and validation for CV
X_train_val_scaled = np.vstack([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

# Perform cross-validation
cv_scores_lr = cross_val_score(
    lr, X_train_val_scaled, y_train_val, 
    cv=kf, scoring='neg_root_mean_squared_error'
)
cv_scores_knn = cross_val_score(
    knn, X_train_val_scaled, y_train_val, 
    cv=kf, scoring='neg_root_mean_squared_error'
)

print(f"\nLinear Regression:")
print(f"  CV RMSE (mean): {-cv_scores_lr.mean():.4f}")
print(f"  CV RMSE (std):  {cv_scores_lr.std():.4f}")
print(f"  Fold scores: {[-score for score in cv_scores_lr]}")

print(f"\nKNN Regressor:")
print(f"  CV RMSE (mean): {-cv_scores_knn.mean():.4f}")
print(f"  CV RMSE (std):  {cv_scores_knn.std():.4f}")
print(f"  Fold scores: {[-score for score in cv_scores_knn]}")



In [ ]:
# Cell 15: Visualization 1 - Cross-Validation Error per Fold (Loss Curve)
print("\n" + "="*70)
print("VISUALIZATION 1: CROSS-VALIDATION ERROR PER FOLD")
print("="*70)

fold_errors_lr = [-score for score in cv_scores_lr]
fold_errors_knn = [-score for score in cv_scores_knn]

plt.figure(figsize=(10, 6))
plt.plot(range(1, 6), fold_errors_lr, marker='o', linewidth=2, 
         markersize=8, label='Linear Regression', color='blue')
plt.plot(range(1, 6), fold_errors_knn, marker='s', linewidth=2, 
         markersize=8, label='KNN', color='green')
plt.xlabel('Fold Number', fontsize=12)
plt.ylabel('Validation RMSE', fontsize=12)
plt.title('Cross-Validation Error per Fold (Loss Curve Equivalent)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 6))
plt.tight_layout()
plt.show()

print("This plot shows the validation error for each fold in cross-validation")
print("Lower values indicate better performance. Consistent values across folds indicate stable model.")


In [ ]:

# Cell 16: Visualization 2 - Residual Plots (Confusion Matrix Equivalent)
print("\n" + "="*70)
print("VISUALIZATION 2: RESIDUAL PLOTS")
print("="*70)

# Calculate residuals
residuals_lr = y_test - y_pred_lr
residuals_knn = y_test - y_pred_knn

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear Regression residuals
axes[0].scatter(y_pred_lr, residuals_lr, alpha=0.6, color='blue', edgecolors='k', linewidth=0.5)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Values', fontsize=11)
axes[0].set_ylabel('Residuals (Actual - Predicted)', fontsize=11)
axes[0].set_title('Linear Regression: Residual Plot', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# KNN residuals
axes[1].scatter(y_pred_knn, residuals_knn, alpha=0.6, color='green', edgecolors='k', linewidth=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Values', fontsize=11)
axes[1].set_ylabel('Residuals (Actual - Predicted)', fontsize=11)
axes[1].set_title('KNN: Residual Plot', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Residual plots show prediction errors. Good models have:")
print("  - Residuals randomly scattered around zero line")
print("  - No clear patterns in residuals")



In [ ]:

# Cell 17: Visualization 3 - Actual vs Predicted (ROC Curve Equivalent)
print("\n" + "="*70)
print("VISUALIZATION 3: ACTUAL VS PREDICTED VALUES")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear Regression
axes[0].scatter(y_test, y_pred_lr, alpha=0.6, color='blue', edgecolors='k', linewidth=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Values', fontsize=11)
axes[0].set_ylabel('Predicted Values', fontsize=11)
axes[0].set_title('Linear Regression: Actual vs Predicted', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# KNN
axes[1].scatter(y_test, y_pred_knn, alpha=0.6, color='green', edgecolors='k', linewidth=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Values', fontsize=11)
axes[1].set_ylabel('Predicted Values', fontsize=11)
axes[1].set_title('KNN: Actual vs Predicted', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Points closer to the red dashed line indicate better predictions")
print("Perfect predictions would lie exactly on the diagonal line")



In [ ]:
# Cell 18: Visualization 4 - KNN Hyperparameter Tuning (K vs Error)
print("\n" + "="*70)
print("VISUALIZATION 4: KNN HYPERPARAMETER TUNING")
print("="*70)

# Test different K values
k_values = range(1, 21)
k_errors = []

for k in k_values:
    knn_temp = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn_temp, X_train_val_scaled, y_train_val, 
                             cv=5, scoring='neg_root_mean_squared_error')
    k_errors.append(-scores.mean())

plt.figure(figsize=(10, 6))
plt.plot(k_values, k_errors, marker='o', linewidth=2, markersize=8, color='green')
plt.axvline(x=5, color='r', linestyle='--', linewidth=2, label='Selected K=5')
plt.xlabel('Number of Neighbors (K)', fontsize=12)
plt.ylabel('Cross-Validation RMSE', fontsize=12)
plt.title('KNN Hyperparameter Tuning: K vs Validation Error', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.tight_layout()
plt.show()

optimal_k = k_values[np.argmin(k_errors)]
print(f"\nOptimal K value: {optimal_k} (RMSE: {min(k_errors):.4f})")
print(f"Selected K value: 5 (RMSE: {k_errors[4]:.4f})")
print("Lower K values may overfit, higher K values may underfit")



In [ ]:
# Cell 19: Visualization 5 - Model Performance Comparison
print("\n" + "="*70)
print("VISUALIZATION 5: MODEL PERFORMANCE COMPARISON")
print("="*70)

metrics = ['MAE', 'RMSE', 'R²']
lr_scores = [
    mean_absolute_error(y_test, y_pred_lr),
    np.sqrt(mean_squared_error(y_test, y_pred_lr)),
    r2_score(y_test, y_pred_lr)
]
knn_scores = [
    mean_absolute_error(y_test, y_pred_knn),
    np.sqrt(mean_squared_error(y_test, y_pred_knn)),
    r2_score(y_test, y_pred_knn)
]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, lr_scores, width, label='Linear Regression', 
               alpha=0.8, color='blue', edgecolor='black')
bars2 = ax.bar(x + width/2, knn_scores, width, label='KNN', 
               alpha=0.8, color='green', edgecolor='black')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison on Test Set', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=11)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("For MAE and RMSE: Lower is better (less error)")
print("For R²: Higher is better (closer to 1 means better fit)")



In [ ]:
# Cell 20: Final Summary
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)

print("\n📊 Project: Bank Loan Approval Prediction")
print("="*70)

print("\n1. Dataset Summary:")
print(f"   - Original samples: {len(df)}")
print(f"   - Balanced samples: {len(df_balanced)} (after downsampling)")
print(f"   - Training: {len(X_train)} samples (60%)")
print(f"   - Validation: {len(X_val)} samples (20%)")
print(f"   - Testing: {len(X_test)} samples (20%)")
print(f"   - Features: {X.shape[1]}")
print(f"   - Class balance: 50/50 (No Loan / Loan)")

print("\n2. Models Implemented:")
print("   - Linear Regression")
print("   - K-Nearest Neighbors (K=5)")

print("\n3. Best Model on Test Set:")
if r2_score(y_test, y_pred_lr) > r2_score(y_test, y_pred_knn):
    print("   🏆 Linear Regression")
    print(f"   - R² Score: {r2_score(y_test, y_pred_lr):.4f}")
    print(f"   - RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.4f}")
else:
    print("   🏆 KNN Regressor")
    print(f"   - R² Score: {r2_score(y_test, y_pred_knn):.4f}")
    print(f"   - RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_knn)):.4f}")

print("\n4. All Required Visualizations:")
print("   ✓ Cross-validation error per fold (loss curve)")
print("   ✓ Residual plots (confusion matrix equivalent)")
print("   ✓ Actual vs predicted plots (ROC curve equivalent)")
print("   ✓ Hyperparameter tuning plot")
print("   ✓ Model performance comparison")

print("\n5. Documentation Completed:")
print("   ✓ Dataset information")
print("   ✓ Feature extraction details")
print("   ✓ Cross-validation setup")
print("   ✓ All hyperparameters documented")
print("   ✓ Results on validation and test sets")

print("\n" + "="*70)
print("✅ ALL REQUIREMENTS FULFILLED")
print("="*70)